In [1]:
#%pip install pyproj

In [2]:
from pyproj import Proj, transform
import math
import os
import pathlib
import geopandas as gpd
from tqdm import tqdm
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

In [3]:


def find_utm_zone(lon, lat):
    utm_band = str((math.floor((lon +  180) /  6) %  60) +  1)
    if len(utm_band) ==  1:
        utm_band = '0' + utm_band
    return utm_band

# Example usage for jamaica:
lon = -77  # Replace with your longitude
lat =  18    # Replace with your latitude
utm_zone = find_utm_zone(lon, lat)
print("UTM Zone: ", utm_zone)


UTM Zone:  18


In [5]:

# Define paths
folder_path = r'P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Elevation_DEM\copDEM_Polygonize\1000_merged'
output_folder = r'P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Elevation_DEM\Final_output'

#Sample of what GeoDF looks like
gdf=gpd.read_file(pathlib.Path().joinpath(folder_path,'a1000_merged.gpkg'))
gdf

: 

In [4]:
# Define paths
folder_path = r'C:\Users\rowe\Desktop\test'
output_folder = r'P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\02_Exposure\Elevation_DEM\Final_output'

#Sample of what GeoDF looks like
gdf=gpd.read_file(pathlib.Path().joinpath(folder_path,'a3.gpkg'))
gdf

,elev,geometry
0,9,"POLYGON ((43.88653 1.00014, 43.88653 0.99986, ..."
1,9,"POLYGON ((43.88736 1.00014, 43.88736 0.99986, ..."
2,7,"POLYGON ((43.88764 1.00014, 43.88764 0.99986, ..."
3,6,"POLYGON ((43.88792 1.00014, 43.88792 0.99986, ..."
4,3,"POLYGON ((43.88819 1.00014, 43.88819 0.99986, ..."
...,...,...
21600,5,"POLYGON ((43.00014 0.12153, 43.00014 0.12125, ..."
21601,2,"POLYGON ((43.00042 0.12153, 43.00042 0.12125, ..."
21602,3,"POLYGON ((42.99986 0.12125, 42.99986 0.12069, ..."
21603,2,"POLYGON ((43.00014 0.12097, 43.00014 0.12069, ..."


In [5]:
# Create an empty DataFrame to store the results
output_csv = pd.DataFrame(columns=['lon','lat','elev', 'area'])
output_csv

,lon,lat,elev,area


In [9]:



# Assuming lon and lat are the longitude and latitude of the centroid of the GeoDataFrame
centroid = gdf.geometry.centroid.iloc[0]
lon, lat = centroid.x, centroid.y

# Find the UTM zone
utm_zone = find_utm_zone(lon, lat)

# Define the EPSG code for the UTM zone
epsg_code = int(f"326{utm_zone}")

# Reproject to the UTM zone
gdf_utm = gdf.to_crs(epsg=epsg_code)
# gdf_utm=gdf

# Add area column in square meters
gdf_utm['area_sqm'] = gdf_utm['geometry'].area



for index, row in gdf_utm.iterrows():
    #output_csv =output_csv.append({'lon': lon, 'lat':lat,'elev': row['elev'], 'area': row['area']}, ignore_index=True)
    output_csv = pd.concat([output_csv, pd.DataFrame([{'lon': lon, 'lat': lat, 'elev': row['elev'], 'area': row['area_sqm']}])], ignore_index=True)
output_csv

,lon,lat,elev,area
0,43.886806,1.0,9,1.543210e-07
1,43.886806,1.0,9,7.716049e-08
2,43.886806,1.0,7,7.716049e-08
3,43.886806,1.0,6,7.716049e-08
4,43.886806,1.0,3,7.716049e-08
...,...,...,...,...
86415,43.886806,1.0,5,9.501771e+02
86416,43.886806,1.0,2,9.501768e+02
86417,43.886806,1.0,3,3.800709e+03
86418,43.886806,1.0,2,9.501771e+02


In [ ]:
#coutput concatenated cv
output_csv.to_csv(pathlib.Path().joinpath(output_folder,'copDEM_final_csv_jk.csv'))